In [6]:
import sys
sys.path.append( "/home/peer/PycharmProjects/neslearng")
sys.path.append( "/home/peer/PycharmProjects/neslearng/extern/nes-async-rl/")

In [50]:
import pandas
import hsa.ba.rewards as rewards
import itertools
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
import hsa.machine_constants
from hsa.nes_python_input import py_to_nes_wrapper

import numpy as np
import matplotlib.pyplot as plt
import itertools
import os.path

In [14]:
from dqn_phi import dqn_phi
import nes 
from a3c_nes import A3CLSTM
import chainer
from chainer import serializers

In [83]:
example_model_path = "/home/peer/studium/bachelor/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5" 
rom_path = hsa.machine_constants.mario_rom_location
nr_of_attempts = 50

In [81]:
base_paht = "/home/peer/studium/bachelor/checkpoints/{}/{}"
model_names = [
    "scaled_fine_main_reward_1"
    ,"ehrenbrav_1 a3c_4"
    ,"nes_a3c_scaled_finer_main_reward_1"
    ,"scaled_finer_main_reward_with_points_1"
]
model_chepoint_combos = [(model_name,"16000000_finish.h5")for model_name in model_names]
model_chepoint_combos.append(("ehrenbrav_1 a3c_4","9000000.h5"))
model_chepoint_combos.append(("scaled_fine_main_reward_1","9400000.h5"))
model_chepoint_combos.append(("nes_a3c_scaled_finer_main_reward_1","9200000.h5"))
#model_chepoint_combos = itertools.product(model_names, chepoint_filenames) 
model_snapshots = {(directory,checkpoint):base_paht.format(directory,checkpoint) 
                   for (directory,checkpoint) 
                   in model_chepoint_combos}
# make sure every file exists
for path in model_snapshots.values():
    if not os.path.isfile(path):
        print(path)

In [82]:
model_snapshots

{('ehrenbrav_1 a3c_4',
  '16000000_finish.h5'): '/home/peer/studium/bachelor/checkpoints/ehrenbrav_1 a3c_4/16000000_finish.h5',
 ('ehrenbrav_1 a3c_4',
  '9000000.h5'): '/home/peer/studium/bachelor/checkpoints/ehrenbrav_1 a3c_4/9000000.h5',
 ('nes_a3c_scaled_finer_main_reward_1',
  '16000000_finish.h5'): '/home/peer/studium/bachelor/checkpoints/nes_a3c_scaled_finer_main_reward_1/16000000_finish.h5',
 ('nes_a3c_scaled_finer_main_reward_1',
  '9200000.h5'): '/home/peer/studium/bachelor/checkpoints/nes_a3c_scaled_finer_main_reward_1/9200000.h5',
 ('scaled_fine_main_reward_1',
  '16000000_finish.h5'): '/home/peer/studium/bachelor/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5',
 ('scaled_fine_main_reward_1',
  '9400000.h5'): '/home/peer/studium/bachelor/checkpoints/scaled_fine_main_reward_1/9400000.h5',
 ('scaled_finer_main_reward_with_points_1',
  '16000000_finish.h5'): '/home/peer/studium/bachelor/checkpoints/scaled_finer_main_reward_with_points_1/16000000_finish.h5'}

In [37]:
#n_actions = nes.NES(hsa.machine_constants.mario_rom_location).number_of_actions

def load_model(weights_path):
    n_actions = nes.n_actions
    model = A3CLSTM(n_actions)
    serializers.load_hdf5(weights_path, model)
    return model
example_model = load_model(example_model_path)

In [22]:
def make_eval_functions():
    return {
        "main_reward": rewards.make_finer_main_reward(),
        "points": rewards.make_delta_points(),
        "time_left": rewards.reward_for_time_left
    }

In [32]:
def eval_performance(rom, model, deterministic=False,
                     record_screen_path=None):
    nesApi = NESInterface(rom)
    env = nes.NES(rom,outside_nes_interface=nesApi,record_screen_path=record_screen_path)
    model.reset_state()
    ehrenbrav_sum = 0
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        ehrenbrav_sum += env.receive_action(a)
        
        # TODO Sum of Ehrenbrav, Sum of Main Reward, sum of time left, did it beat the level?, how many deaths
        ram = nesApi.getRAM()
        for name, reward_function in other_rewards.items():
            sums[name] += reward_function(ram)
    
    sums["ehrenbrav"] = ehrenbrav_sum
    return sums

In [38]:
eval_performance(hsa.machine_constants.mario_rom_location,example_model,deterministic=False)

{'ehrenbrav': 14405, 'main_reward': 7.890625, 'points': 40.0, 'time_left': 0}

In [54]:
eval_results = {}
for (experiment_name,model_name), weights in model_snapshots.items():
    model = load_model(weights)
    for attempt_nr in range(nr_of_attempts):
        eval_results[(experiment_name,model_name,attempt_nr)] = eval_performance(rom_path,model,deterministic=False)

In [57]:
pandas.DataFrame.from_dict(eval_results,orient="index")

main_reward  ehrenbrav  \
ehrenbrav_1 a3c_4         16000000_finish.h5 0     6.093750       9810   
                                             1     6.097656      10295   
scaled_fine_main_reward_1 16000000_finish.h5 0     6.050781      13520   
                                             1     7.898438      15970   

                                                points  time_left  
ehrenbrav_1 a3c_4         16000000_finish.h5 0    20.0          0  
                                             1    50.0          0  
scaled_fine_main_reward_1 16000000_finish.h5 0    40.0          0  
                                             1    60.0          0

In [56]:
pandas.DataFrame.from_dict?